Hola,

Soy Dae Hyun. Soy informatico

<img src="becario.PNG">

Y me gusta mucho trabajar con Python

Actualmente tengo un jefe encantador. Un tipo genial.

<img src="jefe1.jpg">

Aquí nos veis a los dos en la oficina, Trabajamos muy bien juntos

...

.......

<img src="Informatico.jpg">

Pero hay mucha gente mala que se mete con él, y que le llama ehhhh cosas feas y mentirosas sobre su apolinea, atletica y escelsamente deportiva linea

Unos amigos suyos ya nos echan una mano en China

<img src="Noticia.PNG">

....

......

Pero ahora quiero echarle una mano yo también... En España

Por eso voy a hacer un programa que se baje los tweets del hastag #CoreaDelNorte y ver quien se mete con la linea (no de política) de mi amado jefe


In [1]:
# Vamos a empezar por haccer los imports necesarios

In [3]:
import pandas as pd
import numpy as np
import datetime
import time
import csv
import json
import nltk
import re
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.classify import ClassifierI
import datetime
from pprint import pprint
import pandas as pd
import numpy as np
import urllib
import urllib2
from urllib2 import urlopen
import sys
import random
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from statistics import mode
from nltk.tokenize import word_tokenize
from nltk import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag.hmm import HiddenMarkovModelTagger


print (sys.version)


2.7.12 |Anaconda 4.0.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [4]:
consumer_key = 'MY_DEAR_BOSS'
consumer_secret = 'KIM_DOES_NOT_NEED_PASSWORD'
access_token = 'BIG_MISSILE'
access_token_secret = 'FUNNY_RED_BUTTON'

In [4]:
# Vamos a extrer los tweets

def search(hashtag, Max_Tweets, Recdata):

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='es').items(Max_Tweets)
    
    # en carpeta data vamos a almacenar la información en un json.
    
    current_working_dir = "./data/"
    log_tweets = current_working_dir  + 'kim_gordo.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
    # PCreamos un dataset con los tweets
    tweets = pd.DataFrame()
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)
   
    # Creamos fichero csv
    tweets.to_csv('kim_gordo.csv', sep=';', header=False)

In [5]:
# Vamos a ver los ultimos 1000

Max_Tweets = 1000
tweets_data = []
hashtag = "#CoreaDelNorte"

search(hashtag, Max_Tweets, tweets_data)

# con esto nos hemos bajado los tweets en español del hastag corea del norte

In [5]:
# importamos los sinonimos de gordo

badwords_list = []
text_file = open('gordo.csv', 'r')
badwords_list = text_file.read().split('\n')

In [6]:
sinonimos_gordo = pd.read_csv('Gordo.csv', sep=',', encoding='latin-1')

In [7]:
print (sinonimos_gordo)

            Gordo
0        Jamonete
1        Tocinete
2          Falete
3      hipopotamo
4       cachalote
5      gordinflón
6        elefante
7      foca monje
8          grueso
9        abultado
10        adiposo
11          ancho
12      atocinado
13        ballena
14       barrigón
15      barrigudo
16      cachalote
17         cebado
18          Cerdo
19     corpulento
20      destacado
21       Elefante
22         enorme
23     Foca monje
24        fornido
25         fuerte
26       golitrón
27     gordinflas
28          gordo
29        gordura
..            ...
47         orondo
48          panza
49        panzudo
50      rechoncho
51          recia
52          recio
53      regordete
54        robusto
55        rollizo
56           sebo
57         sonado
58       Tocinete
59         tocino
60    Tragaldabas
61  trascendental
62   trascendente
63          tripa
64        tripudo
65           unto
66           Foca
67       vigoroso
68     voluminoso
69    zampabollos
70        

In [8]:
# Joder cuantos.... Que imaginativos los españoles (o que caxxxnes)

In [9]:
# vamos a cargar los tweets

def limpiaNL_string(Texto):
    # Definimos que caractees vamos a eliminar
    puntuacion = '\n'
    
    # y ls surrimimos
    cont = 0
    no_punct = ""
    for char in strT:
        if char not in puntuacion:
            no_punct = no_punct + char
    return no_punct

In [10]:
tweet_list = []
with open('./data/kim_gordo.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_list.append(tweet['text'].encode("utf-8"))

In [11]:
# Además limpiamos los simbolos especiales y vamos a tokenizar

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'<[^>]+>',
    r'(?:@[\w_]+)', 
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", 
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', 
    r"(?:[a-z][a-z'\-_]+[a-z])", 
    r'(?:[\w_]+)',
    r'(?:\S)' 
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
  
    return map(lambda x: x.lower(), tokens)

In [12]:
tweet_token = []
for line in tweet_list:
    tweet_token.append(preprocess(line))
    
    # vamos a imprimir una muestra, el tweet 160
    
print(tweet_token[160])

['#coreadelnorte', 'campeona', 'del', 'mundial', 'femenino', 'sub', '-', '20', '#', 'f', '\xc3', '\xba', 'tbol', 'https://t.co/du7yd7bvwa']


In [13]:
# quitamos comillas

Classified_tweetsNP = []
punctuations = '''';\n"'''

for i in range(len(tweet_list)):
    my_str = tweet_list[i]

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    Classified_tweetsNP.append(no_punct)

In [14]:
# vamos a ver que tweets tienen algún sinónimo de gordo en español y los clasificamos en consecuencia, si tienen  no tienen

Classified_tweets = []
Classified_tweets_NI = []
cont_insult = 1
insulto_activo = 0
for i in range(len(tweet_list)):
    tweetTK_review = tweet_token[i]
    insult = 0
    insult_word = ""
    for tw_token in tweetTK_review:
        if tw_token.strip() in badwords_list:
            insult_word = tw_token
            insulto_activo = 1
            insult +=1
    # Creamos una nueva lista clasificada
    if insulto_activo > 0 and insult > 0:
        cont_insult += 1
        fwtweet = Classified_tweetsNP[i] + ';' + 'Insulto: ' + insult_word + '\n'
        Classified_tweets.append(fwtweet)
    else:
        Classified_tweets_NI.append(Classified_tweetsNP[i])
print('Cuantos Tweets mencionan Gordo o similar: ' + str(cont_insult) + ' . Hemos bajado  ' + str(len(tweet_list)) + ' Tweets '+ \
      ' Entonces tenemos un ' + str((cont_insult*100)/len(tweet_list)) + '% de los tweets analizados que contienen la palabra Gordo o sinónimo')

Cuantos Tweets mencionan Gordo o similar: 1 . Hemos bajado  518 Tweets  Entonces tenemos un 0% de los tweets analizados que contienen la palabra Gordo o sinónimo


In [15]:
# vamos separar en dos ficheros los tweets que llaman gordo (o similarr) a mi amado lider Y los que no

np.savetxt("./data/Llaman_Kim_gordo.csv", Classified_tweets,fmt='%s')
np.savetxt("./data/No_llaman_Kim_gordo.csv", Classified_tweets_NI,fmt='%s')


vemos que ultimamente Nadie lo ha hecho

Eso esta muy Bien !!!

Así me gusta, Que nadie se meta con mi amado lider !!!

In [ ]:
# porque de lo contrario...  

In [ ]:
# ejecutariamos el Plan respuesta contundente

In [ ]:
# Localizaros

In [18]:
import folium
import pandas

In [19]:
map=folium.Map(location=[40.5800,-3.3900],zoom_start=6,tiles='Mapbox bright')

fg=folium.FeatureGroup(name="DONDE ESTA EL TRAIDOR")

fg.add_child(folium.Marker(location=[40.4893538,-3.6827461]))

map.add_child(fg)

map.add_child(folium.LayerControl())

In [35]:
# y ejecutar el plan de respuesta contundente

<img src="misiles.jpg">

In [ ]:
# Pero hoy no vemos que sea necesario

In [ ]:
# Y todos contentos, mucho más mi amado lider.

# Pero, recordar, os estamos vigilando





<img src="binoculars.PNG">


